In [1]:
%load_ext autoreload
%autoreload 2

from timeit import default_timer
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

from riptide import (
    TimeSeries, ffa_search, SubIntegrations, find_peaks
    )

In [8]:
time_series = TimeSeries.from_presto_inf('/home/vince/work/pulsars/presto_time_series/J1855+0307_DM400.00.inf')
print(time_series)

start = default_timer()
ts, plan, pgram = ffa_search(time_series, period_min=0.5, period_max=4.0, bins_min=240, bins_max=260, rmed_width=4.0)
end = default_timer()
print('Time: {0:.2f} ms'.format(1.0e3 * (end - start)))
pgram.display()

TimeSeries {nsamp = 8754411, tsamp = 6.4000e-05, tobs = 560.282}
Time: 664.49 ms


In [9]:
start = default_timer()
peaks, stats_tracker, polyco_tracker = find_peaks(
    pgram,
    segment_dftbins_length=40.0, 
    min_segments=8, 
    snr_min=7.0, 
    nsigma=7.0, 
    peak_clustering_radius=0.20, 
    polydeg=2
    )
end = default_timer()
print('Time: {0:.2f} ms'.format(1.0e3 * (end - start)))

peaks = sorted(peaks, key=lambda p: p.period, reverse=False)
for ii, peak in enumerate(peaks):
    print('{0:3d} {1!s}'.format(ii, peak))

Time: 122.30 ms
  0 Peak [P0 = 5.635766983e-01, W =   6, S/N =   9.50]
  1 Peak [P0 = 8.453498483e-01, W =   4, S/N =  18.45]
  2 Peak [P0 = 1.268013358e+00, W =   3, S/N =   7.49]
  3 Peak [P0 = 1.690691590e+00, W =   3, S/N =  11.91]
  4 Peak [P0 = 2.536031723e+00, W =   2, S/N =   9.39]
  5 Peak [P0 = 3.381401062e+00, W =   1, S/N =   8.26]
  6 Peak [P0 = 3.741457701e+00, W =  28, S/N =   9.42]


In [10]:
peak = peaks[1]
plt.figure(figsize=(12, 6), dpi=75)
plt.subplot(211)
plt.imshow(
    peak.snr_trials.T, aspect='auto', cmap=plt.cm.Greys
    )
plt.yticks(range(pgram.widths.size), pgram.widths)

plt.subplot(212)
dp = (peak.period_trials - peak.period) * 1e6
plt.plot(dp, peak.snr_trials.max(axis=1))
plt.xlim(dp[0], dp[-1])
plt.xlabel('Delta Period (us)')
plt.grid(linestyle=':')
plt.tight_layout()
plt.show()

In [11]:
SubIntegrations.from_time_series(ts, peak.period, nbins=256, nsubs=128).display()